# 0 TorchText

## Dataset Preview

Your first step to deep learning in NLP. We will be mostly using PyTorch. Just like torchvision, PyTorch provides an official library, torchtext, for handling text-processing pipelines. 

We will be using previous session tweet dataset. Let's just preview the dataset.

In [43]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [44]:
file = "/content/drive/My Drive/tweets.csv"

In [45]:
import pandas as pd
df = pd.read_csv(file)
df.head()

,tweets,labels
0,Obama has called the GOP budget social Darwini...,1
1,"In his teen years, Obama has been known to use...",0
2,IPA Congratulates President Barack Obama for L...,0
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0
4,RT @wardollarshome: Obama has approved more ta...,1


In [46]:
df.shape

(1364, 2)

In [47]:
df.labels.value_counts()

0    931
1    352
2     81
Name: labels, dtype: int64

## Defining Fields

Now we shall be defining LABEL as a LabelField, which is a subclass of Field that sets sequen tial to False (as it’s our numerical category class). TWEET is a standard Field object, where we have decided to use the spaCy tokenizer and convert all the text to lower‐ case.

In [105]:
# Import Library
import random
import torch, torchtext
from torchtext import data 

# Manual Seed
SEED = 43
torch.manual_seed(SEED)

In [144]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [145]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [213]:
example = [data.Example.fromlist([df.tweets[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [214]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [215]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [216]:
(len(train), len(valid))

(1159, 205)

An example from the dataset:

In [217]:
vars(train.examples[10])

{'labels': 0,
 'tweets': ['Obama',
  ',',
  'Romney',
  'agree',
  ':',
  'Admit',
  'women',
  'to',
  'Augusta',
  'golf',
  'club',
  ':',
  'US',
  'President',
  'Barack',
  'Obama',
  'believes',
  'women',
  'should',
  'be',
  'allowe',
  '...',
  'http://t.co/PVKrepqI']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [218]:
Tweet.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [219]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)
print(Tweet.vocab.stoi)

Size of input vocab :  4651
Size of label vocab :  3
Top 10 words appreared repeatedly : [('Obama', 1069), (':', 783), ('#', 780), ('.', 761), (',', 598), ('"', 550), ('the', 542), ('RT', 516), ('?', 419), ('to', 400)]
Labels :  defaultdict(<function _default_unk_index at 0x7f58507d0598>, {0: 0, 1: 1, 2: 2})
defaultdict(<function _default_unk_index at 0x7f58507d0598>, {'<unk>': 0, '<pad>': 1, 'Obama': 2, ':': 3, '#': 4, '.': 5, ',': 6, '"': 7, 'the': 8, 'RT': 9, '?': 10, 'to': 11, "'s": 12, 'of': 13, 'a': 14, 'you': 15, 'is': 16, '!': 17, 'with': 18, '-': 19, 'in': 20, 'on': 21, 'and': 22, 'I': 23, 'kid': 24, 'for': 25, '...': 26, 'he': 27, 'that': 28, 'WhatsRomneyHiding': 29, 'from': 30, 'do': 31, 'like': 32, ' ': 33, 'his': 34, 'Romney': 35, 'have': 36, 'President': 37, 'You': 38, 'So': 39, 'go': 40, "'re": 41, 'American': 42, 'Do': 43, 'at': 44, 'has': 45, 'cool': 46, "'": 47, '&': 48, 'British': 49, 'Mcdonalds': 50, 'Ohhh': 51, 'UK': 52, 'Queen': 53, 'tea': 54, '@ohgirlphrase': 55,

**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [220]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [221]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [222]:
import os, pickle
with open('s7_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [223]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                           bidirectional=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)  # one to actually execute padding
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num, directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [224]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)

embedding_dim = 300

num_hidden_nodes = 100

num_output_nodes = 3

num_layers = 2

dropout = 0.2


# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [225]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(4651, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 1,958,803 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [226]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=4e-5)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)  # max returns the max values and max indices, here we need only the max indices
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [227]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)  # return average loss and average acc

**Evaluation Loop**

In [228]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [229]:
N_EPOCHS = 100
best_valid_loss = float('inf')  # initialize with float

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 's7_saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.089 | Train Acc: 44.84%
	 Val. Loss: 1.078 |  Val. Acc: 56.25% 

	Train Loss: 1.072 | Train Acc: 57.38%
	 Val. Loss: 1.059 |  Val. Acc: 62.50% 

	Train Loss: 1.055 | Train Acc: 62.62%
	 Val. Loss: 1.042 |  Val. Acc: 65.62% 

	Train Loss: 1.039 | Train Acc: 65.07%
	 Val. Loss: 1.023 |  Val. Acc: 68.75% 

	Train Loss: 1.023 | Train Acc: 67.10%
	 Val. Loss: 1.007 |  Val. Acc: 70.09% 

	Train Loss: 1.007 | Train Acc: 68.79%
	 Val. Loss: 0.991 |  Val. Acc: 70.98% 

	Train Loss: 0.991 | Train Acc: 69.88%
	 Val. Loss: 0.976 |  Val. Acc: 71.43% 

	Train Loss: 0.977 | Train Acc: 70.98%
	 Val. Loss: 0.963 |  Val. Acc: 70.54% 

	Train Loss: 0.962 | Train Acc: 70.81%
	 Val. Loss: 0.950 |  Val. Acc: 70.09% 

	Train Loss: 0.948 | Train Acc: 70.64%
	 Val. Loss: 0.938 |  Val. Acc: 69.20% 

	Train Loss: 0.935 | Train Acc: 70.22%
	 Val. Loss: 0.927 |  Val. Acc: 69.20% 

	Train Loss: 0.922 | Train Acc: 70.22%
	 Val. Loss: 0.917 |  Val. Acc: 69.20% 

	Train Loss: 0.910 | Train Acc: 70.22%
	

## Model Testing

In [68]:
#load weights and tokenizer

path='./s7_saved_weights.pt'
model.load_state_dict(torch.load(path));
model.eval();
tokenizer_file = open('./s7_tokenizer.pkl', 'rb')
tokenizer = pickle.load(tokenizer_file)

#inference 

import spacy
nlp = spacy.load('en')

def classify_tweet(tweet):
    
    categories = {0: "Negative", 1:"Positive", 2:"Neutral"}
    
    # tokenize the tweet 
    tokenized = [tok.text for tok in nlp.tokenizer(tweet)] 
    # convert to integer sequence using predefined tokenizer dictionary
    indexed = [tokenizer[t] for t in tokenized]        
    # compute no. of words        
    length = [len(indexed)]  # i.e length of incoming text
    # convert to tensor                                    
    tensor = torch.LongTensor(indexed).to(device)   
    # reshape in form of batch, no. of words           
    tensor = tensor.unsqueeze(1).T  
    # convert to tensor                          
    length_tensor = torch.LongTensor(length)
    # Get the model prediction                  
    prediction = model(tensor, length_tensor)

    _, pred = torch.max(prediction, 1) 
    
    return categories[pred.item()]

In [69]:
device

device(type='cuda')

In [70]:
classify_tweet("A valid explanation for why Trump won't let women on the golf course.")

'Positive'

## Discussion on Data Augmentation Techniques 

You might wonder exactly how you can augment text data. After all, you can’t really flip it horizontally as you can an image! :D 

In contrast to data augmentation in images, augmentation techniques on data is very specific to final product you are building. As its general usage on any type of textual data doesn't provides a significant performance boost, that's why unlike torchvision, torchtext doesn’t offer a augmentation pipeline. Due to powerful models as transformers, augmentation tecnhiques are not so preferred now-a-days. But its better to know about some techniques with text that will provide your model with a little more information for training. 

### Synonym Replacement

First, you could replace words in the sentence with synonyms, like so:

    The dog slept on the mat

could become

    The dog slept on the rug

Aside from the dog's insistence that a rug is much softer than a mat, the meaning of the sentence hasn’t changed. But mat and rug will be mapped to different indices in the vocabulary, so the model will learn that the two sentences map to the same label, and hopefully that there’s a connection between those two words, as everything else in the sentences is the same.

### Random Insertion
A random insertion technique looks at a sentence and then randomly inserts synonyms of existing non-stopwords into the sentence n times. Assuming you have a way of getting a synonym of a word and a way of eliminating stopwords (common words such as and, it, the, etc.), shown, but not implemented, in this function via get_synonyms() and get_stopwords(), an implementation of this would be as follows:


In [71]:
def random_insertion(sentence, n): 
    words = remove_stopwords(sentence) 
    for _ in range(n):
        new_synonym = get_synonyms(random.choice(words))
        sentence.insert(randrange(len(sentence)+1), new_synonym) 
    return sentence

## Random Deletion
As the name suggests, random deletion deletes words from a sentence. Given a probability parameter p, it will go through the sentence and decide whether to delete a word or not based on that random probability. Consider of it as pixel dropouts while treating images.

In [72]:
def random_deletion(words, p=0.2): 
    if len(words) == 1: # return if single word
        return words[0]
    remaining = list(filter(lambda x: random.uniform(0,1) > p,words)) 
    if len(remaining) == 0: # if not left, sample a random word
        return random.choice(words)
    else:
        return ''.join(remaining)

### Random Swap
The random swap augmentation takes a sentence and then swaps words within it n times, with each iteration working on the previously swapped sentence. Here we sample two random numbers based on the length of the sentence, and then just keep swapping until we hit n.

In [73]:
def random_swap(sentence, n=3):
    if isinstance(sentence, str):
        sentence = list(sentence) 
    length = range(len(sentence)) 
    for _ in range(n):
        idx1, idx2 = random.sample(length, 2)
        sentence[idx1], sentence[idx2] = sentence[idx2], sentence[idx1] 
    return "".join(sentence)

For more on this please go through this [paper](https://arxiv.org/pdf/1901.11196.pdf).

### Back Translation

Another popular approach for augmenting text datasets is back translation. This involves translating a sentence from our target language into one or more other languages and then translating all of them back to the original language. We can use the Python library googletrans for this purpose. 

In [74]:
# !pip install googletrans

In [75]:
import random
import googletrans
from googletrans import Translator

translator = Translator()
sentence = ['The dog slept on the rug']

available_langs = list(googletrans.LANGUAGES.keys()) 

trans_lang = random.choice(available_langs) 
print(f"Translating to {googletrans.LANGUAGES[trans_lang]}")

translations = translator.translate(sentence, dest=trans_lang) 
t_text = [t.text for t in translations]
print(t_text)

translations_en_random = translator.translate(t_text, src=trans_lang, dest='en') 
en_text = [t.text for t in translations_en_random]
print(en_text)

ModuleNotFoundError: ignored

# Runs with Text Augmentation


In [80]:
df['delete'] = df['tweets'].apply(random_deletion)

In [81]:
df.head()

,tweets,labels,delete
0,Obama has called the GOP budget social Darwini...,1,Obm as caled the GO bug social awinism. Nice t...
1,"In his teen years, Obama has been known to use...",0,"I his teen yeas, Obam habeen known to usemarju..."
2,IPA Congratulates President Barack Obama for L...,0,IPA Cngratulate Prdet Brack Oma for Leadership...
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0,T @Profesr_Why: #WhatsRmneHin hisconnection t ...
4,RT @wardollarshome: Obama has approved more ta...,1,RTwardollarome: Obama ha appovd ore targee ass...


In [82]:
df['swap'] = df['tweets'].apply(random_swap)

In [83]:
df.head()

,tweets,labels,delete,swap
0,Obama has called the GOP budget social Darwini...,1,Obm as caled the GO bug social awinism. Nice t...,Obama hashcalled t e GOP budget sicial Darwini...
1,"In his teen years, Obama has been known to use...",0,"I his teen yeas, Obam habeen known to usemarju...","In hi steen yeaus, Obama has been known to use..."
2,IPA Congratulates President Barack Obama for L...,0,IPA Cngratulate Prdet Brack Oma for Leadership...,IPA Congratulates President Barack ObaIa for L...
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0,T @Profesr_Why: #WhatsRmneHin hisconnection t ...,RT @Professor_Why: #WhatsRomneyHidingh- his co...
4,RT @wardollarshome: Obama has approved more ta...,1,RTwardollarome: Obama ha appovd ore targee ass...,RT @waedollarshome: Obama has approved more ta...


In [84]:
df['combined_aug'] = df['tweets'].apply(random_deletion).apply(random_swap)

In [85]:
df.head()

,tweets,labels,delete,swap,combined_aug
0,Obama has called the GOP budget social Darwini...,1,Obm as caled the GO bug social awinism. Nice t...,Obama hashcalled t e GOP budget sicial Darwini...,Obama has cald he GOP bdgetsoia .arwirism.Nici...
1,"In his teen years, Obama has been known to use...",0,"I his teen yeas, Obam habeen known to usemarju...","In hi steen yeaus, Obama has been known to use...",n hin tenyak Obmahas eesnr ownt usemariuaa and...
2,IPA Congratulates President Barack Obama for L...,0,IPA Cngratulate Prdet Brack Oma for Leadership...,IPA Congratulates President Barack ObaIa for L...,IPA Congraubates PreidenBaac Olma for Laedersh...
3,RT @Professor_Why: #WhatsRomneyHiding - his co...,0,T @Profesr_Why: #WhatsRmneHin hisconnection t ...,RT @Professor_Why: #WhatsRomneyHidingh- his co...,RT Profsor_Wy: #hatsomoyiding -i coection to s...
4,RT @wardollarshome: Obama has approved more ta...,1,RTwardollarome: Obama ha appovd ore targee ass...,RT @waedollarshome: Obama has approved more ta...,R twardllarshome: O ma as apoved more targeted...


# DELETE

In [248]:
Tweet = data.Field(sequential = True, tokenize = 'spacy', batch_first =True, include_lengths=True)
Label = data.LabelField(tokenize ='spacy', is_target=True, batch_first =True, sequential =False)

Having defined those fields, we now need to produce a list that maps them onto the list of rows that are in the CSV:

In [249]:
fields = [('tweets', Tweet),('labels',Label)]

Armed with our declared fields, lets convert from pandas to list to torchtext. We could also use TabularDataset to apply that definition to the CSV directly but showing an alternative approach too.

In [250]:
example = [data.Example.fromlist([df.delete[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [251]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [252]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [253]:
(len(train), len(valid))

(1159, 205)

An example from the dataset:

In [254]:
vars(train.examples[10])

{'labels': 0,
 'tweets': ['Obama',
  ',',
  'Rmney',
  'agree',
  ':',
  'dmit',
  'omen',
  'touus',
  'of',
  'club',
  ':',
  'USPesident',
  'Baack',
  'Obaabeieves',
  'women',
  'shod',
  'be',
  'llowe',
  '..',
  'ttp:/.o',
  '/',
  'VKrepq']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [255]:
Tweet.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [256]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)
print(Tweet.vocab.stoi)

Size of input vocab :  9227
Size of label vocab :  3
Top 10 words appreared repeatedly : [(':', 616), ('.', 587), ('#', 499), (',', 462), ('"', 363), ('?', 290), ('RT', 264), (' ', 246), ('Obama', 243), ('/', 219)]
Labels :  defaultdict(<function _default_unk_index at 0x7f58507d0598>, {0: 0, 1: 1, 2: 2})
defaultdict(<function _default_unk_index at 0x7f58507d0598>, {'<unk>': 0, '<pad>': 1, ':': 2, '.': 3, '#': 4, ',': 5, '"': 6, '?': 7, 'RT': 8, ' ': 9, 'Obama': 10, '/': 11, 'the': 12, 'o': 13, 'to': 14, 'a': 15, '!': 16, "'s": 17, "'": 18, '-': 19, 'of': 20, 'he': 21, 'is': 22, 'i': 23, 'I': 24, 't': 25, 'T': 26, '..': 27, 'on': 28, 'you': 29, 'Obam': 30, 'R': 31, 'in': 32, 'Obma': 33, 'th': 34, '...': 35, 'e': 36, 'n': 37, '&': 38, 'and': 39, 'bama': 40, 'Obaa': 41, 'kid': 42, 's': 43, 'Oama': 44, 'h': 45, 'go': 46, 'te': 47, 'with': 48, 'So': 49, 'for': 50, 'his': 51, 'd': 52, 'at': 53, 'like': 54, 'UK': 55, 'do': 56, 'it': 57, 'Do': 58, 'have': 59, 'Oba': 60, 'an': 61, 'that': 62, '

**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [257]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [258]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [259]:
import os, pickle
with open('s7_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [268]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                           bidirectional=False)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)  # one to actually execute padding
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num, directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [269]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)

embedding_dim = 300

num_hidden_nodes = 100

num_output_nodes = 3

num_layers = 2

dropout = 0.2


# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [270]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(9227, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 3,010,003 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [271]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=4e-5)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)  # max returns the max values and max indices, here we need only the max indices
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [272]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)  # return average loss and average acc

**Evaluation Loop**

In [273]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [274]:
N_EPOCHS = 100
best_valid_loss = float('inf')  # initialize with float

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 's7_saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.097 | Train Acc: 35.53%
	 Val. Loss: 1.084 |  Val. Acc: 46.05% 

	Train Loss: 1.085 | Train Acc: 47.59%
	 Val. Loss: 1.073 |  Val. Acc: 57.83% 

	Train Loss: 1.073 | Train Acc: 55.19%
	 Val. Loss: 1.060 |  Val. Acc: 59.17% 

	Train Loss: 1.060 | Train Acc: 58.36%
	 Val. Loss: 1.047 |  Val. Acc: 60.51% 

	Train Loss: 1.047 | Train Acc: 59.71%
	 Val. Loss: 1.034 |  Val. Acc: 62.95% 

	Train Loss: 1.033 | Train Acc: 62.67%
	 Val. Loss: 1.021 |  Val. Acc: 66.52% 

	Train Loss: 1.020 | Train Acc: 64.78%
	 Val. Loss: 1.007 |  Val. Acc: 68.30% 

	Train Loss: 1.006 | Train Acc: 66.72%
	 Val. Loss: 0.995 |  Val. Acc: 68.75% 

	Train Loss: 0.992 | Train Acc: 67.31%
	 Val. Loss: 0.981 |  Val. Acc: 68.75% 

	Train Loss: 0.976 | Train Acc: 68.58%
	 Val. Loss: 0.967 |  Val. Acc: 68.75% 

	Train Loss: 0.961 | Train Acc: 69.17%
	 Val. Loss: 0.954 |  Val. Acc: 68.75% 

	Train Loss: 0.945 | Train Acc: 69.34%
	 Val. Loss: 0.940 |  Val. Acc: 68.30% 

	Train Loss: 0.929 | Train Acc: 69.34%
	

# SWAP

In [275]:
example = [data.Example.fromlist([df.swap[i],df.labels[i]], fields) for i in range(df.shape[0])] 

In [276]:
# Creating dataset
#twitterDataset = data.TabularDataset(path="tweets.csv", format="CSV", fields=fields, skip_header=True)

twitterDataset = data.Dataset(example, fields)

Finally, we can split into training, testing, and validation sets by using the split() method:

In [277]:
(train, valid) = twitterDataset.split(split_ratio=[0.85, 0.15], random_state=random.seed(SEED))

In [278]:
(len(train), len(valid))

(1159, 205)

An example from the dataset:

In [279]:
vars(train.examples[10])

{'labels': 0,
 'tweets': ['Obama',
  ',',
  'Romney',
  'agree',
  ':',
  'AdmiS',
  'women',
  'no',
  'Augusta',
  'golf',
  'club',
  ':',
  'Ut',
  'Presidett',
  'Barack',
  'Obama',
  'believes',
  'women',
  'should',
  'be',
  'allowe',
  '...',
  'http://q.co/PVKreptI']}

## Building Vocabulary

At this point we would have built a one-hot encoding of each word that is present in the dataset—a rather tedious process. Thankfully, torchtext will do this for us, and will also allow a max_size parameter to be passed in to limit the vocabu‐ lary to the most common words. This is normally done to prevent the construction of a huge, memory-hungry model. We don’t want our GPUs too overwhelmed, after all. 

Let’s limit the vocabulary to a maximum of 5000 words in our training set:


In [280]:
Tweet.build_vocab(train)
Label.build_vocab(train)

By default, torchtext will add two more special tokens, <unk> for unknown words and <pad>, a padding token that will be used to pad all our text to roughly the same size to help with efficient batching on the GPU.

In [281]:
print('Size of input vocab : ', len(Tweet.vocab))
print('Size of label vocab : ', len(Label.vocab))
print('Top 10 words appreared repeatedly :', list(Tweet.vocab.freqs.most_common(10)))
print('Labels : ', Label.vocab.stoi)
print(Tweet.vocab.stoi)

Size of input vocab :  8242
Size of label vocab :  3
Top 10 words appreared repeatedly : [(':', 820), ('Obama', 750), ('.', 720), ('#', 718), (',', 592), ('"', 512), ('RT', 435), ('the', 424), ('?', 390), (' ', 383)]
Labels :  defaultdict(<function _default_unk_index at 0x7f58507d0598>, {0: 0, 1: 1, 2: 2})
defaultdict(<function _default_unk_index at 0x7f58507d0598>, {'<unk>': 0, '<pad>': 1, ':': 2, 'Obama': 3, '.': 4, '#': 5, ',': 6, '"': 7, 'RT': 8, 'the': 9, '?': 10, ' ': 11, 'to': 12, "'s": 13, 'a': 14, 'of': 15, '!': 16, 'you': 17, '-': 18, 'is': 19, '/': 20, 'with': 21, 'on': 22, 'I': 23, 'in': 24, 'and': 25, 'kid': 26, '...': 27, 'for': 28, 'he': 29, 'do': 30, 'like': 31, 'his': 32, "'": 33, 'that': 34, 'from': 35, 'So': 36, 'have': 37, '&': 38, 'go': 39, 'You': 40, 'UK': 41, 'at': 42, 'Do': 43, 'tea': 44, 'Ohhh': 45, 'cool': 46, "'re": 47, 'Romney': 48, 'has': 49, 'it': 50, 'British': 51, 'not': 52, 'President': 53, 'Queen': 54, 'American': 55, 'can': 56, 'The': 57, 'we': 58, 'b

**Lots of stopwords!!**

Now we need to create a data loader to feed into our training loop. Torchtext provides the BucketIterator method that will produce what it calls a Batch, which is almost, but not quite, like the data loader we used on images.

But at first declare the device we are using.

In [282]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [283]:
train_iterator, valid_iterator = data.BucketIterator.splits((train, valid), batch_size = 32, 
                                                            sort_key = lambda x: len(x.tweets),
                                                            sort_within_batch=True, device = device)

Save the vocabulary for later use

In [284]:
import os, pickle
with open('s7_tokenizer.pkl', 'wb') as tokens: 
    pickle.dump(Tweet.vocab.stoi, tokens)

## Defining Our Model

We use the Embedding and LSTM modules in PyTorch to build a simple model for classifying tweets.

In this model we create three layers. 
1. First, the words in our tweets are pushed into an Embedding layer, which we have established as a 300-dimensional vector embedding. 
2. That’s then fed into a 2 stacked-LSTMs with 100 hidden features (again, we’re compressing down from the 300-dimensional input like we did with images). We are using 2 LSTMs for using the dropout.
3. Finally, the output of the LSTM (the final hidden state after processing the incoming tweet) is pushed through a standard fully connected layer with three outputs to correspond to our three possible classes (negative, positive, or neutral).

In [292]:
import torch.nn as nn
import torch.nn.functional as F

class classifier(nn.Module):
    
    # Define all the layers used in model
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, dropout):
        
        super().__init__()          
        
        # Embedding layer
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
        # LSTM layer
        self.encoder = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           dropout=dropout,
                           batch_first=True,
                           bidirectional=True)
        # try using nn.GRU or nn.RNN here and compare their performances
        # try bidirectional and compare their performances
        
        # Dense layer
        self.fc = nn.Linear(hidden_dim, output_dim)
        
    def forward(self, text, text_lengths):
        
        # text = [batch size, sent_length]
        embedded = self.embedding(text)
        # embedded = [batch size, sent_len, emb dim]
      
        # packed sequence
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.cpu(), batch_first=True)  # one to actually execute padding
        
        packed_output, (hidden, cell) = self.encoder(packed_embedded)
        #hidden = [batch size, num layers * num directions,hid dim]
        #cell = [batch size, num layers * num directions,hid dim]
    
        # Hidden = [batch size, hid dim * num, directions]
        dense_outputs = self.fc(hidden)   
        
        # Final activation function softmax
        output = F.softmax(dense_outputs[0], dim=1)
            
        return output

In [293]:
# Define hyperparameters
size_of_vocab = len(Tweet.vocab)

embedding_dim = 300

num_hidden_nodes = 100

num_output_nodes = 3

num_layers = 2

dropout = 0.2


# Instantiate the model
model = classifier(size_of_vocab, embedding_dim, num_hidden_nodes, num_output_nodes, num_layers, dropout = dropout)

In [294]:
print(model)

#No. of trianable parameters
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
    
print(f'The model has {count_parameters(model):,} trainable parameters')

classifier(
  (embedding): Embedding(8242, 300)
  (encoder): LSTM(300, 100, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (fc): Linear(in_features=100, out_features=3, bias=True)
)
The model has 3,036,103 trainable parameters


## Model Training and Evaluation

First define the optimizer and loss functions

In [295]:
import torch.optim as optim

# define optimizer and loss
optimizer = optim.Adam(model.parameters(), lr=4e-5)
criterion = nn.CrossEntropyLoss()

# define metric
def binary_accuracy(preds, y):
    #round predictions to the closest integer
    _, predictions = torch.max(preds, 1)  # max returns the max values and max indices, here we need only the max indices
    
    correct = (predictions == y).float() 
    acc = correct.sum() / len(correct)
    return acc
    
# push to cuda if available
model = model.to(device)
criterion = criterion.to(device)

The main thing to be aware of in this new training loop is that we have to reference `batch.tweets` and `batch.labels` to get the particular fields we’re interested in; they don’t fall out quite as nicely from the enumerator as they do in torchvision.

**Training Loop**

In [296]:
def train(model, iterator, optimizer, criterion):
    
    # initialize every epoch 
    epoch_loss = 0
    epoch_acc = 0
    
    # set the model in training phase
    model.train()  
    
    for batch in iterator:
        
        # resets the gradients after every batch
        optimizer.zero_grad()   
        
        # retrieve text and no. of words
        tweet, tweet_lengths = batch.tweets   
        
        # convert to 1D tensor
        predictions = model(tweet, tweet_lengths).squeeze()  
        
        # compute the loss
        loss = criterion(predictions, batch.labels)        
        
        # compute the binary accuracy
        acc = binary_accuracy(predictions, batch.labels)   
        
        # backpropage the loss and compute the gradients
        loss.backward()       
        
        # update the weights
        optimizer.step()      
        
        # loss and accuracy
        epoch_loss += loss.item()  
        epoch_acc += acc.item()    
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)  # return average loss and average acc

**Evaluation Loop**

In [297]:
def evaluate(model, iterator, criterion):
    
    # initialize every epoch
    epoch_loss = 0
    epoch_acc = 0

    # deactivating dropout layers
    model.eval()
    
    # deactivates autograd
    with torch.no_grad():
    
        for batch in iterator:
        
            # retrieve text and no. of words
            tweet, tweet_lengths = batch.tweets
            
            # convert to 1d tensor
            predictions = model(tweet, tweet_lengths).squeeze()
            
            # compute loss and accuracy
            loss = criterion(predictions, batch.labels)
            acc = binary_accuracy(predictions, batch.labels)
            
            # keep track of loss and accuracy
            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

**Let's Train and Evaluate**

In [298]:
N_EPOCHS = 100
best_valid_loss = float('inf')  # initialize with float

for epoch in range(N_EPOCHS):
     
    # train the model
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    
    # evaluate the model
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
    
    # save the best model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 's7_saved_weights.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}% \n')

	Train Loss: 1.090 | Train Acc: 42.74%
	 Val. Loss: 1.093 |  Val. Acc: 40.45% 

	Train Loss: 1.078 | Train Acc: 58.28%
	 Val. Loss: 1.081 |  Val. Acc: 50.45% 

	Train Loss: 1.066 | Train Acc: 61.57%
	 Val. Loss: 1.068 |  Val. Acc: 54.91% 

	Train Loss: 1.053 | Train Acc: 64.61%
	 Val. Loss: 1.055 |  Val. Acc: 58.04% 

	Train Loss: 1.040 | Train Acc: 66.47%
	 Val. Loss: 1.041 |  Val. Acc: 58.48% 

	Train Loss: 1.026 | Train Acc: 67.82%
	 Val. Loss: 1.028 |  Val. Acc: 60.71% 

	Train Loss: 1.013 | Train Acc: 68.41%
	 Val. Loss: 1.014 |  Val. Acc: 60.27% 

	Train Loss: 0.998 | Train Acc: 68.79%
	 Val. Loss: 1.001 |  Val. Acc: 61.16% 

	Train Loss: 0.983 | Train Acc: 69.88%
	 Val. Loss: 0.988 |  Val. Acc: 61.16% 

	Train Loss: 0.969 | Train Acc: 69.97%
	 Val. Loss: 0.976 |  Val. Acc: 63.39% 

	Train Loss: 0.953 | Train Acc: 70.22%
	 Val. Loss: 0.964 |  Val. Acc: 65.62% 

	Train Loss: 0.938 | Train Acc: 69.63%
	 Val. Loss: 0.952 |  Val. Acc: 66.07% 

	Train Loss: 0.923 | Train Acc: 69.80%
	

# **OBSERVATIONS**


---



# Normal without text augmentation
## *lr=4e-5*
### for 50 epochs --> Simple LSTM --> tr:84%, valid:77%
### for 50 epochs --> Bi-directional LSTM --> tr:85.90%, valid:77.68%


## *lr=2e-4*
### for 50 epochs --> Simple LSTM --> tr:92%, valid:83%
### for 100 epochs --> Bi-directional LSTM --> tr:92%.74, valid:77.23%


## *lr=6e-5*
### for 50 epochs --> Bi-directional LSTM --> tr:90%, valid:80%


---




---


-------------------------------------------


---



# With text augmentation
## *lr=4e-5 and 'swap'* 
### for 100 epochs --> Simple LSTM --> tr:92.53%, valid:56.25%
### for 100 epochs --> Bi-directional LSTM --> tr:91.47%, valid:73.21%

## *lr=4e-5 and 'delete'* 
### for 100 epochs --> Simple LSTM --> tr:92.48%, valid:66.96%
### for 100 epochs --> Bi-directional LSTM --> tr:91.98%, valid:67.41%

## *lr=4e-5 and 'combined_aug'* 
### for 100 epochs --> Simple LSTM --> tr:84%, valid:77%
### for 100 epochs --> Bi-directional LSTM --> tr:92.94%, valid:44.27%


